In [16]:
ls

In [17]:
from utils1 import read_txt_file, load_json_lines

In [18]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [19]:
def get_completion(prompt, model="gpt-3.5-turbo-1106"):  # gpt-3.5-turbo
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
        seed=20240407
    )
    return response.choices[0].message["content"]

In [ ]:
# def get_completion(prompt, model="gpt-3.5-turbo-1106"):  # gpt-3.5-turbo
#     messages = [{"role": "user", "content": prompt}]
#     response = openai.ChatCompletion.create(
#         model=model,
#         messages=messages,
#         temperature=0.7,
#         seed=20240407
#     )
#     return response.choices[0].message["content"]

In [22]:
prompt_org = read_txt_file('prompt_org.txt')
print(prompt_org)

In [24]:
product_data_list = load_json_lines('phone_products_20240406_203410.json')
print(len(product_data_list))

In [25]:
product_data = str(product_data_list[-1])

In [ ]:
import json

def merge_questions_created(questions_created_new, questions_created_old):
    # Load the JSON strings into dictionaries
    questions_created_old_js = json.loads(questions_created_old)
    questions_created_new_js = json.loads(questions_created_new)

    # Merge the dictionaries
    questions_created_merged_new_old = {
        "product_id": questions_created_old_js["product_id"],
        "url": questions_created_old_js["url"],
        "questions": questions_created_old_js["questions"] + questions_created_new_js["questions"]
    }

    # Convert the merged dictionary back to a JSON string
    questions_created_merged_new_old = json.dumps(questions_created_merged_new_old, ensure_ascii=False, indent=4)
    
    return questions_created_merged_new_old

In [ ]:
def is_valid_json(json_str):
    
    try:
        json.loads(json_str)
        return True
    except json.JSONDecodeError:
        return False


In [ ]:
from IPython.display import HTML

from datetime import datetime,timedelta

def get_current_timestamp():

    current_time_utc = datetime.utcnow()
    time_difference = timedelta(hours=7)
    
    current_time_hanoi = current_time_utc + time_difference
    timestamp = current_time_hanoi.strftime("%Y%m%d_%H%M%S")
    
    return timestamp

get_current_timestamp()

In [ ]:


for i, product_data in enumerate(product_data_list[:]):
    print(i)
    print(product_data['url'])
    prompt_org = prompt_org = read_txt_file('prompt_org.txt')
    
    prompt_org = prompt_org.replace('{product_data}', str(product_data))
    questions_created_old = 'EMPTY'
    
    for j in range(2):
        if j == 0:
            questions_created_old = 'EMPTY'
            
        prompt_complete = prompt_org.replace('{questions_created}', questions_created_old)
        
        

        questions_created_new = get_completion(prompt_complete)
        while is_valid_json(questions_created_new) == 0:
            questions_created_new = get_completion(prompt_complete)
            
        print(questions_created_new)
            
        if j ==0: 
            questions_created_old = questions_created_new
        else:
            questions_created_old = merge_questions_created(questions_created_new, questions_created_old)
            
        
    
    filename = f"qna_{product_data['product_id']}_{get_current_timestamp()}.json"
    
    with open(filename, "w") as json_file:
        json_file.write(questions_created_old)
    
#     print(f"Saved {filename}.")
                        
    # Generate HTML code for displaying a link to download the file
    download_link_html = f'<a href="{filename}" download>Download "{filename}" here.</a>'

    # Display the HTML code
    display(HTML(download_link_html))

    


In [ ]:
import os
import zipfile

def zip_files_with_prefix(directory, prefix, zip_filename):
    """
    Zip files with a specific prefix in a directory into a zip archive.

    Args:
        directory (str): Directory containing the files.
        prefix (str): Prefix of the files to be zipped.
        zip_filename (str): Name of the zip archive to be created.
    """
    # Get the list of files in the directory
    files_to_zip = [filename for filename in os.listdir(directory) if filename.startswith(prefix)]

    zip_filename = zip_filename.replace('.zip', f'_{len(files_to_zip)}.zip')
    
    # Create a zip file
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        # Add each file with the prefix to the zip archive
        for filename in files_to_zip:
            filepath = os.path.join(directory, filename)
            zipf.write(filepath, filename)

# Directory containing the files
directory = '.'

# Prefix of the files to be zipped
prefix = 'qna'

# Name of the zip archive to be created
zip_filename = 'qna_laptop.zip'

# Zip files with the specified prefix
zip_files_with_prefix(directory, prefix, zip_filename)